In [8]:
import requests, datetime, time, json
from bs4 import BeautifulSoup


res=requests.get('https://www.ptt.cc/bbs/Food/M.1457229157.A.1B3.html')
soup=BeautifulSoup(res.text)




def request_what(board,indexnum):
    index_url='https://www.ptt.cc/bbs/'+str(board)+'/index'+str(indexnum)+'.html'
    index_res=requests.get(index_url)
    return BeautifulSoup(index_res.text)


def grap_push_info(soup):
    pushInfo_list = []
    for i in soup.select('.push'):
        if len(i.select('.push-ipdatetime')[0].string.strip()) > 11:
            continue
        pushInfo_list.append(
                              # date (e.g. '02/18')
                             [i.select('.push-ipdatetime')[0].string.strip(), 
                              # userid (e.g. 'comdan66')
                              i.select('.push-userid')[0].string.strip(), 
                              # push content
                              i.select('.push-content')[0].string[1:].strip()]
        )
    return pushInfo_list

def grap_orig_post(soup):
    return (
        # date (e.g. 'Mon Aug 29 14:32:25 2005')
        soup.select('.article-meta-value')[3].text.strip() +'``!@'+
        # title
        soup.select('.article-meta-value')[2].text.strip() +'``!@'+
        # userid (e.g. 'skyboy')
        soup.select('.article-meta-value')[0].text.split('(')[0].strip() +'``!@'+                   
        # content
        soup.select('#main-content')[0].contents[4].strip()
    ).split('``!@')


def get_year(soup, push_m, push_d):
    m_dict = {'jan':1, 'feb':2, 'mar':3, 'apr':4, 'may':5, 'jun':6, 
              'jul':7, 'aug':8, 'sep':9, 'oct':10, 'nov':11, 'dec':12}
    
    post_y = int(soup.select('.article-meta-value')[3].text.strip()[-4:])
    post_m = m_dict[soup.select('.article-meta-value')[3].text.strip()[4:7].lower()]
    
    if push_m < post_m:
        return str(post_y+1)
    else:
        return str(post_y)

def link(board,indexnum):
    link_list=[]
    for links in request_what(board,indexnum).select('.title'):
        link_list.append('https://www.ptt.cc/bbs/'+links.select('a')[0]['href'])
    return link_list


def crawl_in_page(board, indexNum):
    for linkss in link(board, indexNum):
        try:
            con_res = requests.get(linkss)
            con_soup = BeautifulSoup(con_res.text)
        except:
            counter = 1
            while counter <= 3:
                print('Retry after 2 sec...')
                time.sleep(2)
                con_res = requests.get(linkss)
                con_soup = BeautifulSoup(con_res.text)
                counter += 1
        try:
            # crawl main content
            ptt_data.append({
                'date'   : grap_orig_post(con_soup)[0],
                'title'  : grap_orig_post(con_soup)[1],
                'userid' : grap_orig_post(con_soup)[2],
                'content': grap_orig_post(con_soup)[3],
                'url'    : con_res.url,
                'is_orig_post': True
            })

            # crawl push content
            for i in grap_push_info(con_soup):

                ptt_data.append({
                    'date'   : get_year(con_soup, int(i[0][0:2]), int(i[0][3:5])) + '/' + i[0],
                    'title'  : grap_orig_post(con_soup)[1],
                    'userid' : i[1],
                    'content': i[2],
                    'url'    : con_res.url,
                    'is_orig_post': False
                })
        
        except:
            continue





